In [ ]:
import numpy as np
import pandas as pd
from miles_edit import *

travel = pd.read_csv('output_2023_09_05/reports/travel-4-combined.csv')
segments = pd.read_csv('output_2023_09_05/reports/overall-5-ways.csv')
intersections = pd.read_csv('output_2023_09_05/reports/overall-6-intersections.csv')
reach = pd.read_csv('output_2023_09_05/reports/overall-2-reach-type.csv')

In [ ]:
## Don't remember why I did this but interesting to look at later maybe
reach[reach["Type"] =="project total"].sort_values(by="Intersections count").reset_index().plot(y=["Intersections count",'Segments count'])

### Projects to test manually:
~~More connected/less crossings
64ff64a08cf244c982529b2f
65454061b5bfdcb11540ec4f
More long/linear with many crossings
65284187772d22a2108ddc4c
64921e2f1930d10600997fd9
NVM these don't work, probably inputted after Matt sent this data. I only have up to 64e...~~


long/linear

64921e2f1930d10600997fd9 ### ERROR ### No volume even though intersections are selected???? 5/7 + 4/6 + 5/7 + 4/7 + 5/7 + 5/7 = 28/41

64b0406741e08c5dff327a1f 79/106 selected

connected

645582371c8b985d1be43a07 19/41

64dfb34bcb0d64389a5ea11e 14/28

~~For some reason when I try to run this to test it out I get division by 0...
Maybe there are some additional errors caused by too much rounding?
NVM the rounding was removed in the final implementation of the code.~~

In [ ]:
#Pull data and input into the function from the volume to miles code
def miles(project_id,mode,adjacent_ratio): # **NOTE: Changed the labels in volume_to_miles.json to be "bicycling" and "walking" for consistency so I only have to use one variable :P
    proj_reach = reach[reach["Project ID"] == project_id][reach["Type"] == "project total"]
    volume = travel[travel["Project ID"] == project_id][travel["M Mode"] == mode][travel["K Estimate"] == "mean"]["Existing Travel"]
    distance = calculate_distance(mode, proj_reach.iloc[0,4], proj_reach.iloc[0,2], volume, adjacent_ratio)
    print("The total distance travelled is: "+ str(distance)+ " miles")
miles("64dfb34bcb0d64389a5ea11e","walking",(14/28))